<a href="https://colab.research.google.com/github/bharathts1507/TSAI-Assignments-EVA4/blob/master/TSAI_EVA4_S5_Model_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Method 3:  Use image augmentation, GAP layer and optimized steps LRstep - s10

#Target : 
- Lesser parameters around 10K
- Test Accuracy ~99%
- ~15 Epochs 

#Results: 
- Parameters used : 12716
- Train Accuracy: 99.07%
- Test Accuracy : 98.72%

#Analysis - Observation :
- Less number of parameters requires more epochs to reach higher accuracy
- Could achieve higher accuracy with more parameters with few epochs.

"""

'\nMethod 3:  Use image augmentation, GAP layer and optimized steps LRstep - s10\n\n#Target : \n- Lesser parameters around 10K\n- Test Accuracy ~99%\n- ~15 Epochs \n\n#Results: \n- Parameters used : 12716\n- Train Accuracy: 99.07%\n- Test Accuracy : 98.72%\n\n#Analysis - Observation :\n- Less number of parameters requires more epochs to reach higher accuracy\n- Could achieve higher accuracy with more parameters with few epochs.\n\n'

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values. 
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])


In [4]:
train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


In [5]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

CUDA Available? False


In [0]:
import torch.nn.functional as F
dropout_value = 0.1
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 26

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(dropout_value)
        ) # output_size = 24

        # TRANSITION BLOCK 1
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
        ) # output_size = 24
        self.pool1 = nn.MaxPool2d(2, 2) # output_size = 12

        # CONVOLUTION BLOCK 2
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 10
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 8
        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 6
        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),            
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value)
        ) # output_size = 6
        
        # OUTPUT BLOCK
        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=6)
        ) # output_size = 1

        self.convblock8 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
            # nn.BatchNorm2d(10),
            # nn.ReLU(),
            # nn.Dropout(dropout_value)
        ) 


        self.dropout = nn.Dropout(dropout_value)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        x = self.convblock6(x)
        x = self.convblock7(x)
        x = self.gap(x)        
        x = self.convblock8(x)

        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [7]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
           Dropout-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 32, 24, 24]           4,608
              ReLU-6           [-1, 32, 24, 24]               0
       BatchNorm2d-7           [-1, 32, 24, 24]              64
           Dropout-8           [-1, 32, 24, 24]               0
            Conv2d-9           [-1, 10, 24, 24]             320
        MaxPool2d-10           [-1, 10, 12, 12]               0
           Conv2d-11           [-1, 16, 10, 10]           1,440
             ReLU-12           [-1, 16, 10, 10]               0
      BatchNorm2d-13           [-1, 16, 10, 10]              32
          Dropout-14           [-1,

In [0]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [9]:
from torch.optim.lr_scheduler import StepLR

model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=6, gamma=0.1)


EPOCHS = 20
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    # scheduler.step()
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]

EPOCH: 0


Loss=0.22088882327079773 Batch_id=937 Accuracy=91.47: 100%|██████████| 938/938 [01:59<00:00,  7.85it/s]
Loss=0.04781715199351311 Batch_id=0 Accuracy=98.44:   0%|          | 1/938 [00:00<02:18,  6.78it/s]


Test set: Average loss: 0.0607, Accuracy: 9827/10000 (98.27%)

EPOCH: 1


Loss=0.051382843405008316 Batch_id=937 Accuracy=97.78: 100%|██████████| 938/938 [01:58<00:00,  7.92it/s]
Loss=0.13030415773391724 Batch_id=0 Accuracy=98.44:   0%|          | 1/938 [00:00<02:00,  7.78it/s]


Test set: Average loss: 0.0358, Accuracy: 9886/10000 (98.86%)

EPOCH: 2


Loss=0.012150265276432037 Batch_id=937 Accuracy=98.13: 100%|██████████| 938/938 [01:58<00:00,  7.89it/s]
Loss=0.0089182173833251 Batch_id=0 Accuracy=100.00:   0%|          | 1/938 [00:00<02:00,  7.78it/s]


Test set: Average loss: 0.0276, Accuracy: 9915/10000 (99.15%)

EPOCH: 3


Loss=0.014543573372066021 Batch_id=937 Accuracy=98.47: 100%|██████████| 938/938 [02:00<00:00,  7.78it/s]
Loss=0.03647299110889435 Batch_id=0 Accuracy=98.44:   0%|          | 1/938 [00:00<02:05,  7.47it/s]


Test set: Average loss: 0.0296, Accuracy: 9899/10000 (98.99%)

EPOCH: 4


Loss=0.0028861246537417173 Batch_id=937 Accuracy=98.56: 100%|██████████| 938/938 [01:59<00:00,  7.86it/s]
Loss=0.004460902884602547 Batch_id=0 Accuracy=100.00:   0%|          | 1/938 [00:00<02:05,  7.48it/s]


Test set: Average loss: 0.0278, Accuracy: 9903/10000 (99.03%)

EPOCH: 5


Loss=0.22065575420856476 Batch_id=937 Accuracy=98.61: 100%|██████████| 938/938 [01:59<00:00,  7.88it/s]
Loss=0.05151268467307091 Batch_id=0 Accuracy=98.44:   0%|          | 1/938 [00:00<02:06,  7.43it/s]


Test set: Average loss: 0.0283, Accuracy: 9907/10000 (99.07%)

EPOCH: 6


Loss=0.12862658500671387 Batch_id=937 Accuracy=98.65: 100%|██████████| 938/938 [01:59<00:00,  7.83it/s]
Loss=0.02411969006061554 Batch_id=0 Accuracy=98.44:   0%|          | 1/938 [00:00<02:01,  7.74it/s]


Test set: Average loss: 0.0223, Accuracy: 9932/10000 (99.32%)

EPOCH: 7


Loss=0.0362081415951252 Batch_id=937 Accuracy=98.72: 100%|██████████| 938/938 [01:59<00:00,  7.84it/s]
Loss=0.00624737236648798 Batch_id=0 Accuracy=100.00:   0%|          | 1/938 [00:00<02:11,  7.15it/s]


Test set: Average loss: 0.0208, Accuracy: 9938/10000 (99.38%)

EPOCH: 8


Loss=0.0468030720949173 Batch_id=937 Accuracy=98.79: 100%|██████████| 938/938 [01:59<00:00,  7.83it/s]
Loss=0.025978432968258858 Batch_id=0 Accuracy=98.44:   0%|          | 1/938 [00:00<02:04,  7.51it/s]


Test set: Average loss: 0.0214, Accuracy: 9930/10000 (99.30%)

EPOCH: 9


Loss=0.014580576680600643 Batch_id=937 Accuracy=98.83: 100%|██████████| 938/938 [01:59<00:00,  7.86it/s]
Loss=0.15965910255908966 Batch_id=0 Accuracy=96.88:   0%|          | 1/938 [00:00<02:04,  7.54it/s]


Test set: Average loss: 0.0208, Accuracy: 9937/10000 (99.37%)

EPOCH: 10


Loss=0.07186883687973022 Batch_id=937 Accuracy=98.86: 100%|██████████| 938/938 [01:59<00:00,  7.88it/s]
Loss=0.013194159604609013 Batch_id=0 Accuracy=100.00:   0%|          | 1/938 [00:00<01:56,  8.04it/s]


Test set: Average loss: 0.0207, Accuracy: 9934/10000 (99.34%)

EPOCH: 11


Loss=0.14160780608654022 Batch_id=937 Accuracy=98.96: 100%|██████████| 938/938 [02:00<00:00,  7.79it/s]
Loss=0.005567013286054134 Batch_id=0 Accuracy=100.00:   0%|          | 1/938 [00:00<02:09,  7.23it/s]


Test set: Average loss: 0.0223, Accuracy: 9933/10000 (99.33%)

EPOCH: 12


Loss=0.003035388421267271 Batch_id=937 Accuracy=98.92: 100%|██████████| 938/938 [01:59<00:00,  7.83it/s]
Loss=0.0039343577809631824 Batch_id=0 Accuracy=100.00:   0%|          | 1/938 [00:00<02:01,  7.69it/s]


Test set: Average loss: 0.0197, Accuracy: 9937/10000 (99.37%)

EPOCH: 13


Loss=0.04313816875219345 Batch_id=937 Accuracy=98.97: 100%|██████████| 938/938 [01:59<00:00,  7.83it/s]
Loss=0.01553802378475666 Batch_id=0 Accuracy=98.44:   0%|          | 1/938 [00:00<02:04,  7.52it/s]


Test set: Average loss: 0.0224, Accuracy: 9932/10000 (99.32%)

EPOCH: 14


Loss=0.0154187036678195 Batch_id=937 Accuracy=98.98: 100%|██████████| 938/938 [01:59<00:00,  7.83it/s]
Loss=0.012881205417215824 Batch_id=0 Accuracy=100.00:   0%|          | 1/938 [00:00<02:06,  7.42it/s]


Test set: Average loss: 0.0211, Accuracy: 9925/10000 (99.25%)

EPOCH: 15


Loss=0.050128478556871414 Batch_id=937 Accuracy=99.00: 100%|██████████| 938/938 [02:01<00:00,  7.74it/s]
Loss=0.16685062646865845 Batch_id=0 Accuracy=95.31:   0%|          | 1/938 [00:00<01:55,  8.08it/s]


Test set: Average loss: 0.0192, Accuracy: 9941/10000 (99.41%)

EPOCH: 16


Loss=0.03386743366718292 Batch_id=937 Accuracy=99.01: 100%|██████████| 938/938 [02:01<00:00,  7.71it/s]
Loss=0.006070686969906092 Batch_id=0 Accuracy=100.00:   0%|          | 1/938 [00:00<02:02,  7.64it/s]


Test set: Average loss: 0.0217, Accuracy: 9927/10000 (99.27%)

EPOCH: 17


Loss=0.002797372406348586 Batch_id=937 Accuracy=99.01: 100%|██████████| 938/938 [01:59<00:00,  7.83it/s]
Loss=0.006677250377833843 Batch_id=0 Accuracy=100.00:   0%|          | 1/938 [00:00<01:58,  7.93it/s]


Test set: Average loss: 0.0200, Accuracy: 9946/10000 (99.46%)

EPOCH: 18


Loss=0.032052163034677505 Batch_id=937 Accuracy=98.98: 100%|██████████| 938/938 [02:01<00:00,  7.74it/s]
Loss=0.05256103724241257 Batch_id=0 Accuracy=98.44:   0%|          | 1/938 [00:00<02:03,  7.57it/s]


Test set: Average loss: 0.0219, Accuracy: 9929/10000 (99.29%)

EPOCH: 19


Loss=0.001763061387464404 Batch_id=937 Accuracy=99.10: 100%|██████████| 938/938 [02:03<00:00,  7.58it/s]



Test set: Average loss: 0.0204, Accuracy: 9943/10000 (99.43%)



Text(0.5, 1.0, 'Test Accuracy')

In [10]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2,2,figsize=(15,10))
axs[0, 0].plot(train_losses)
axs[0, 0].set_title("Training Loss")
axs[1, 0].plot(train_acc[4000:])
axs[1, 0].set_title("Training Accuracy")
axs[0, 1].plot(test_losses)
axs[0, 1].set_title("Test Loss")
axs[1, 1].plot(test_acc)
axs[1, 1].set_title("Test Accuracy")